In [16]:
import cv2
import mediapipe as mp
from datetime import datetime

# Initialize MediaPipe FaceMesh and Pose
mp_face_mesh = mp.solutions.face_mesh

# Initialize the FaceMesh module
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Initialize normal chin position (dagu normal)
normal_chin = None
change_detected = False
change_frame_count = 0
looking_down_timer = None
required_looking_time = 2  # 2 detik

# Capture video from camera
cap = cv2.VideoCapture(0)  # Use 0 for default camera

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Convert BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform face landmark detection using MediaPipe
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            if normal_chin is None:
                # Set normal chin position on first frame
                normal_chin = (face_landmarks.landmark[8].x, face_landmarks.landmark[8].y)

            # Extract coordinates of chin
            chin = (face_landmarks.landmark[8].x, face_landmarks.landmark[8].y)

            # Check for drastic change in chin position
            if abs(chin[1] - normal_chin[1]) > normal_chin[1] * 0.5:  # You can adjust the threshold here
                if change_detected == False:
                    change_frame_count = 120
                change_detected = True
                print(f'kamera berubah\t{abs(chin[1] - normal_chin[1])}\t{normal_chin[1] * 0.5}')
            else:
                change_frame_count = 120
                change_detected = False
            
            # Perform calibration if change was detected
            if change_detected:
                if change_frame_count > 0:
                    change_frame_count -= 1
                    print(f'-===  change_detected: {change_frame_count}  ===-')
                else:
                    normal_chin = chin
                    change_detected = False

            # Compare current chin position with normal chin position
            range_normal_chin = normal_chin[1] + (normal_chin[1] * 0.15)
            if chin[1] > range_normal_chin:
                if looking_down_timer is None:
                    looking_down_timer = datetime.now()
                status = "Looking down"
            else:
                status = "Not looking down"
                looking_down_timer = None

            if looking_down_timer is not None:
                diff_time = (datetime.now() - looking_down_timer).total_seconds()
                cv2.putText(frame, f'{diff_time:.2f} seconds', (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                
                if diff_time > required_looking_time:
                    cv2.putText(frame, 'NGANTUK BOS', (10, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
            print(f'{normal_chin[1]}\t{chin[1]}\t{range_normal_chin}')
            
            # Draw status on the frame
            cv2.putText(frame, status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Draw landmarks on the frame
            for landmark in face_landmarks.landmark:
                x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)

    cv2.imshow("Head Angle Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0.634086012840271	0.634086012840271	0.7291989147663116
0.634086012840271	0.6354345083236694	0.7291989147663116
0.634086012840271	0.635290265083313	0.7291989147663116
0.634086012840271	0.6357294321060181	0.7291989147663116
0.634086012840271	0.6357216238975525	0.7291989147663116
0.634086012840271	0.6357555985450745	0.7291989147663116
0.634086012840271	0.6360073685646057	0.7291989147663116
0.634086012840271	0.6359999775886536	0.7291989147663116
0.634086012840271	0.6362693309783936	0.7291989147663116
0.634086012840271	0.6362364888191223	0.7291989147663116
0.634086012840271	0.6362589001655579	0.7291989147663116
0.634086012840271	0.6362126469612122	0.7291989147663116
0.634086012840271	0.6362717747688293	0.7291989147663116
0.634086012840271	0.6363090872764587	0.7291989147663116
0.634086012840271	0.6364861726760864	0.7291989147663116
0.634086012840271	0.6364759206771851	0.7291989147663116
0.634086012840271	0.6366000175476074	0.7291989147663116
0.634086012840271	0.6367442607879639	0.72919891476

In [17]:
cap.release()
cv2.destroyAllWindows()